# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [4]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [5]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [6]:
# Importing and initialize the linear regression model class
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

# Fit model
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
# Calculate predictions on training and test sets
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [8]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

In [10]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

MSE_train = mean_squared_error(y_train, y_hat_train)
MSE_test = mean_squared_error(y_test, y_hat_test)
print(f'MSE for training: {MSE_train}')
print(f'MSE for test: {MSE_test}')

MSE for training: 21.896705419911832
MSE for test: 21.01128343658588


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [13]:
X

,CHAS,RM,DIS,B,LSTAT
0,0.0,6.575,0.542096,1.000000,-1.275260
1,0.0,6.421,0.623954,1.000000,-0.263711
2,0.0,7.185,0.623954,0.989737,-1.627858
3,0.0,6.998,0.707895,0.994276,-2.153192
4,0.0,7.147,0.707895,1.000000,-1.162114
...,...,...,...,...,...
501,0.0,6.593,0.331081,0.987619,-0.169811
502,0.0,6.120,0.297277,1.000000,-0.274682
503,0.0,6.976,0.274575,1.000000,-1.067939
504,0.0,6.794,0.315551,0.991301,-0.836660


In [15]:
X[5:5+5+1]

,CHAS,RM,DIS,B,LSTAT
5,0.0,6.430,0.707895,0.992990,-1.200048
6,0.0,6.012,0.671500,0.996722,0.248456
7,0.0,6.172,0.700059,1.000000,0.968416
8,0.0,5.631,0.709276,0.974104,1.712312
9,0.0,6.004,0.743201,0.974305,0.779802
10,0.0,6.377,0.727217,0.988956,1.077829


In [34]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    df = pd.DataFrame(data)
    start = 0
    partition_size = len(df) // k
    remainder = len(df) % k
    datasets = []
    i = 0
     
    while start < (len(df) - partition_size):
        if i < remainder: 
            subdataset = df.iloc[start:start+partition_size+1]
            datasets.append(subdataset)
            start += partition_size +1
            i += 1
        else:
            subdataset = df.iloc[start:start+partition_size]
            datasets.append(subdataset)
            start += partition_size
    
    # add 1 to fold size to account for leftovers           
    return datasets

### Apply it to the Boston Housing data

In [37]:
# Make sure to concatenate the data again
bos_data = pd.concat([X, y], axis = 1)

In [38]:
bos_data.head()

,CHAS,RM,DIS,B,LSTAT,target
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [57]:
# Apply kfolds() to bos_data with 5 folds
bos_folds = kfolds(bos_data, 5)

In [58]:
bos_folds[1]

,CHAS,RM,DIS,B,LSTAT,target
102,0.0,6.405,0.369415,0.177720,-0.012136,18.6
103,0.0,6.137,0.369415,0.993873,0.378596,19.3
104,0.0,6.167,0.321174,0.989384,0.235000,20.1
105,0.0,5.851,0.262627,0.992814,0.717270,19.5
106,0.0,5.836,0.282946,0.996898,0.925237,19.5
...,...,...,...,...,...,...
198,0.0,7.274,0.786695,0.988149,-0.801053,34.6
199,0.0,6.975,0.806093,1.000000,-1.422033,34.9
200,0.0,7.135,0.806093,0.968228,-1.462710,32.9
201,0.0,6.162,0.722095,0.992108,-0.608764,24.1


In [63]:
def kfolds2(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 

In [64]:
# Apply kfolds() to bos_data with 5 folds
bos_folds = kfolds2(bos_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [65]:
test_errs = []
train_errs = []
k=5

features = ['CHAS', 'RM', 'DIS', 'B', 'LSTAT']

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    
    # Fit a linear regression model
    linreg.fit(train[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']], train['target'])
    
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    
    MSE_train = mean_squared_error(y_train, y_hat_train)
    MSE_test = mean_squared_error(y_test, y_hat_test)
    
    train_errs.append(MSE_train)
    test_errs.append(MSE_test)

print(train_errs)
print(test_errs)

[22.274062330448356, 22.08148456834344, 23.191140604333533, 24.360229654910885, 23.147803237448006]
[21.01585155079122, 21.263934386193256, 23.21409366394934, 22.040462830096157, 22.821797486268245]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [67]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [68]:
cv_5_results


array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 